### Logistic Regression for Multi-class classification problem

In [453]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Load the data
data = pd.read_csv('iris.csv')
# drop the first column
data = data.drop(data.columns[0], axis=1)
# converting the data into numpy array
data = data.values
# converting the string labels into numbers
for i in range(data.shape[0]):
    if data[i,4] == 'Iris-setosa':
        data[i,4] = 0
    elif data[i,4] == 'Iris-versicolor':
        data[i,4] = 1
    else:
        data[i,4] = 2
# splitting the data into features and labels
X_train, X_test, Y_train, Y_test = train_test_split(data[:,0:4], data[:,4], test_size=0.2, random_state=0, stratify=data[:,4])

In [454]:
# converting the data into float64 and int32 types only to avoid future errors
X_train = X_train.astype(np.float64)
X_test = X_test.astype(np.float64)
Y_train = Y_train.astype(np.int32)
Y_test = Y_test.astype(np.int32)

In [455]:
# Define the sigmoid function
def sigmoid(z):
    z = -z
    exponential = np.exp(z)
    return 1/(1+exponential)

In [456]:
# Define the combine function
def combine(X,weights, constant_weight):
    result = np.dot(X, weights)
    result = result + constant_weight
    return result

In [457]:
# Define the logistic regression function
def logistic_regression(X_train, Y_train, learning_rate, no_of_iterations):
    # add a column of ones to X_train and X_test to account for the constant weight
    X_train = np.hstack((np.ones((X_train.shape[0],1)),X_train))
    # initialising the weight vector of the dimension of the number of features
    weights = np.zeros((X_train.shape[1],1))
    # previous weights to check for convergence, if the difference between the current and previous weights is less than 1e-4, then the algorithm has converged
    prev_weights = np.zeros((X_train.shape[1],1))
    for i in range(no_of_iterations):
        # computing the z_value for the current weights and its logistic value
        z_value = combine(X_train, weights, 0)
        # changing the values of z_value to the a probability value
        Y_predicted = sigmoid(z_value)
        # converting the predicted values into 0 and 1 
        Y_predicted[Y_predicted >= 0.5] = 1
        Y_predicted[Y_predicted < 0.5] = 0
        # computing the error
        error = Y_predicted - Y_train.reshape(Y_train.shape[0],1)
        # computing the gradient
        gradient = X_train.T.dot(error)
        # assign the current weights to the previous weights
        prev_weights = weights
        # updating the weights
        weights = weights - learning_rate * gradient/X_train.shape[0]
        # checking for convergence
        if np.linalg.norm(weights - prev_weights) < 1e-4:
            break
    return weights

In [458]:
# all the labels in the dataset
labels = np.unique(Y_train)
# stores all the weights for the 3 logistic regression models corresponding to the 3 labels
all_weights = np.zeros((len(labels), X_train.shape[1]))
all_constant_weight = np.zeros(len(labels))
# training the 3 logistic regression models
for i in range(len(labels)):
    Y_train_binary = np.zeros((Y_train.shape[0], 1))
    Y_train_binary[Y_train == labels[i]] = 1
    # performing logistic regression for each label 
    weights = logistic_regression(X_train, Y_train_binary, 0.01, 1000)
    all_weights[i] = weights[1:].T
    all_constant_weight[i] = weights[0]

In [459]:
# Predicting the result for the test data using the weights from the logistic regression function 
z_value = combine(X_test, all_weights.T, all_constant_weight)
# converting the z_value into probabilities
Y_predicted = sigmoid(z_value)
# the class with the highest probability is the predicted class
Y_predicted = np.argmax(Y_predicted, axis=1)

In [460]:
# Compute the accuracy
count = 0
for i in range(Y_predicted.shape[0]):
    if Y_predicted[i] == Y_test[i]:
        count += 1
accuracy = count/Y_predicted.shape[0]*100
print('Accuracy of the model on the test data is: ', accuracy, '%')

Accuracy of the model on the test data is:  90.0 %
